# EcoMap

## Setup

### Ecoscope

In [ ]:
ECOSCOPE_RAW = "https://raw.githubusercontent.com/wildlife-dynamics/ecoscope/master"

In [ ]:
%pip install \
    'ecoscope[analysis,mapping,plotting] @ git+https://github.com/wildlife-dynamics/ecoscope@v1.8.26' &> /dev/null

In [ ]:
import os
import sys

import geopandas as gpd
import numpy as np
import pandas as pd
import shapely

import ecoscope
from ecoscope.contrib import geemap
from ecoscope.mapping import EcoMap
from ecoscope.analysis.UD import calculate_etd_range
from ecoscope.analysis.percentile import get_percentile_area
from ecoscope.analysis.astronomy import is_night
from ecoscope.analysis.classifier import apply_color_map

ecoscope.init(silent=True)

### Google Drive Setup

In [ ]:
output_dir = "Ecoscope-Outputs"

if "google.colab" in sys.modules:
    from google.colab import drive

    drive.mount("/content/drive/", force_remount=True)
    output_dir = os.path.join("/content/drive/MyDrive/", output_dir)

os.makedirs(output_dir, exist_ok=True)

### Earth Engine

In [ ]:
import ee

try:
    EE_ACCOUNT = os.getenv("EE_ACCOUNT")
    EE_PRIVATE_KEY_DATA = os.getenv("EE_PRIVATE_KEY_DATA")
    if EE_ACCOUNT and EE_PRIVATE_KEY_DATA:
        geemap.ee_initialize(credentials=ee.ServiceAccountCredentials(EE_ACCOUNT, key_data=EE_PRIVATE_KEY_DATA))
    else:
        geemap.ee_initialize()

except ee.EEException:
    ee.Authenticate()
    geemap.ee_initialize()

## Load sample data

### Vehicle tracks

In [ ]:
ecoscope.io.download_file(
    f"{ECOSCOPE_RAW}/tests/sample_data/vector/KDB025Z.csv",
    os.path.join(output_dir, "KDB025Z.csv"),
)

vehicle_gdf = pd.read_csv(os.path.join(output_dir, "KDB025Z.csv"), index_col="id")
vehicle_gdf["geometry"] = vehicle_gdf["geometry"].apply(lambda x: shapely.wkt.loads(x))
vehicle_gdf = ecoscope.base.Relocations.from_gdf(gpd.GeoDataFrame(vehicle_gdf, crs=4326))
vehicle_gdf = ecoscope.base.Trajectory.from_relocations(vehicle_gdf)

### Elephant Sightings

In [ ]:
ecoscope.io.download_file(
    f"{ECOSCOPE_RAW}/tests/sample_data/vector/elephant_sighting.csv",
    os.path.join(output_dir, "elephant_sighting.csv"),
)

events_df = pd.read_csv(os.path.join(output_dir, "elephant_sighting.csv"), index_col="id")
events_df["geometry"] = events_df["geometry"].apply(lambda x: shapely.wkt.loads(x))
events_gdf = gpd.GeoDataFrame(events_df, crs=4326)

### Regions

In [ ]:
ecoscope.io.download_file(
    f"{ECOSCOPE_RAW}/tests/sample_data/vector/maec_4zones_UTM36S.gpkg",
    os.path.join(output_dir, "maec_4zones_UTM36S.gpkg"),
)

region_gdf = gpd.read_file(os.path.join(output_dir, "maec_4zones_UTM36S.gpkg")).to_crs(4326)

### MoveBank Relocations

In [ ]:
ecoscope.io.download_file(
    f"{ECOSCOPE_RAW}/tests/sample_data/vector/movebank_data.csv",
    os.path.join(output_dir, "movebank_data.csv"),
)

df = pd.read_csv(os.path.join(output_dir, "movebank_data.csv"), index_col=0)
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(x=df["location-long"], y=df["location-lat"]),
    crs=4326,
)
movebank_relocations_gdf = ecoscope.base.Relocations.from_gdf(
    gdf, groupby_col="individual-local-identifier", time_col="timestamp"
)

pnts_filter = ecoscope.base.RelocsCoordinateFilter(
    min_x=-5,
    max_x=1,
    min_y=12,
    max_y=18,
    filter_point_coords=[[180, 90], [0, 0]],
)
movebank_relocations_gdf.apply_reloc_filter(pnts_filter, inplace=True)
movebank_relocations_gdf.remove_filtered(inplace=True)

## EcoMap

### Basic EcoMap

In [ ]:
# Initialize EcoMap by setting the zoom level and center
m = EcoMap(height=800, width=1000, static=False)
m.set_view_state(latitude=0.0236, longitude=37.9062, zoom=6)

# Add tiled base layer
m.add_layer(EcoMap.get_named_tile_layer("OpenStreetMap"))

# Display
m

### Elephant Sighting Map

In [ ]:
m = EcoMap(width=800, height=600)

# Add tiled base layer
m.add_layer(EcoMap.get_named_tile_layer("OpenStreetMap"))

# Set DEM visualization parameters
vis_params = {
    "min": 0,
    "max": 4000,
    "opacity": 0.5,
    "palette": ["006633", "E5FFCC", "662A00", "D8D8D8", "F5F5F5"],
}

# Add Google Earth Engine elevation layer
dem = ee.Image("USGS/SRTMGL1_003")
m.add_layer(m.ee_layer(dem.updateMask(dem.gt(0)), vis_params))

apply_color_map(
    region_gdf,
    "ZoneID",
    ["#7fc97f", "#beaed4", "#fdc086", "#ffff99"],
    output_column_name="zone_colors",
)
m.add_layer(m.polygon_layer(region_gdf, opacity=0.5, fill_color_column="zone_colors"), zoom=True)

# Add trajectory
m.add_layer(m.polyline_layer(vehicle_gdf, get_width=200, get_color="#468af7"))

# Add elephant sighting events
m.add_layer(m.point_layer(events_gdf, get_radius=700, get_fill_color="#f746ad"))

# Add title
m.add_title("Elephant Sighting Map")

# Add north-arrow. Placements are: top-right, top-left, bottom-right, bottom-left
m.add_north_arrow(placement="top-left")

# Add legend
m.add_legend(labels=["KDB025Z_Tracks", "Elephant_Sighting_Events"], colors=["#468af7", "#f746ad"])

# Display
m

### Add local geotiff to a map

In [ ]:
ecoscope.io.download_file(
    f"{ECOSCOPE_RAW}/tests/sample_data/raster/mara_dem.tif",
    os.path.join(output_dir, "mara_dem.tif"),
)

m = EcoMap(width=800, height=600)
m.add_layer(EcoMap.get_named_tile_layer("OpenStreetMap"))
m.add_layer(
    m.geotiff_layer(tiff=os.path.join(output_dir, "mara_dem.tif"), cmap="jet"),
    zoom=True,
)
m

## Day-Night Relocations Map

In [ ]:
movebank_relocations_gdf["is_night"] = is_night(movebank_relocations_gdf.geometry, movebank_relocations_gdf.fixtime)

movebank_relocations_gdf[["groupby_col", "fixtime", "geometry", "is_night"]]

In [ ]:
m = EcoMap(width=800, height=600)

m.add_layer(EcoMap.get_named_tile_layer("OpenStreetMap"))

# Add day_night
apply_color_map(
    movebank_relocations_gdf,
    "is_night",
    ["#292965", "#e7a553"],
    output_column_name="colors",
)
m.add_layer(
    m.point_layer(movebank_relocations_gdf, get_radius=700, fill_color_column="colors"),
    zoom=True,
)

m.add_legend(title="Is Night", labels=["True", "False"], colors=["#292965", "#e7a553"])
m.add_north_arrow(placement="top-left")
m.add_title("Day-Night Relocations")

m

## Day-Night Trajectory Map

In [ ]:
movebank_trajectory_gdf = ecoscope.base.Trajectory.from_relocations(movebank_relocations_gdf)
movebank_traj_seg_filter = ecoscope.base.TrajSegFilter(
    min_length_meters=0.0,
    max_length_meters=float("inf"),
    min_time_secs=0.0,
    max_time_secs=4 * 60 * 60,
    min_speed_kmhr=0.0,
    max_speed_kmhr=10.0,
)
movebank_trajectory_gdf.apply_traj_filter(movebank_traj_seg_filter, inplace=True)

In [ ]:
m = EcoMap(width=800, height=600)

m.add_layer(EcoMap.get_named_tile_layer("OpenStreetMap"))

# Add day_night
apply_color_map(
    movebank_trajectory_gdf,
    "extra__is_night",
    ["#292965", "#e7a553"],
    output_column_name="colors",
)
m.add_layer(
    m.polyline_layer(movebank_trajectory_gdf, get_width=200, color_column="colors"),
    zoom=True,
)

m.add_legend(title="Is Night", labels=["True", "False"], colors=["#292965", "#e7a553"])
m.add_north_arrow(placement="top-left")
m.add_title("Day-Night Relocations")

m

## ETD Range

In [ ]:
raster_profile = ecoscope.io.raster.RasterProfile(
    pixel_size=250.0,  # You may need to reduce grid size if you receive a warning during the calculation
    crs="ESRI:102022",  # Albers Africa Equal Area Conic
    nodata_value=np.nan,
    band_count=1,
)

In [ ]:
def f(trajectory_gdf):
    output_path = os.path.join(output_dir, f"{trajectory_gdf.name}.tif")

    calculate_etd_range(
        trajectory_gdf=trajectory_gdf,
        output_path=output_path,
        max_speed_kmhr=1.05 * trajectory_gdf.speed_kmhr.max(),  # Choose a value above the max recorded segment speed
        raster_profile=raster_profile,
        expansion_factor=1.3,
    )
    return output_path


etd = movebank_trajectory_gdf.groupby("groupby_col").apply(f)

In [ ]:
raster_data = ecoscope.io.raster.RasterData.from_raster_file(etd.at["Salif Keita"])
percentile_areas = get_percentile_area(
    percentile_levels=[50, 60, 70, 80, 90, 99.9],
    raster_data=raster_data,
    subject_id="Salif Keita",
).to_crs(4326)

m = EcoMap(width=800, height=600, static=True)
m.add_layer(EcoMap.get_named_tile_layer("OpenStreetMap"))

apply_color_map(percentile_areas, "percentile", "RdYlGn")

m.add_layer(
    m.polygon_layer(percentile_areas, fill_color_column="percentile_colormap"),
    zoom=True,
)

m.add_north_arrow(placement="top-left")
m.add_title("Salif ETD Range")

m

## Export

### As HTML

In [ ]:
m.to_html(os.path.join(output_dir, "ecomap.html"))